## 模型5 denesenet模型

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

%matplotlib qt

In [2]:
from torch.utils.data import random_split
model_accuracies = {
    'densenet': []
}
# 数据增强和预处理
transform5 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 加载数据集
data_dir = r'D:\深度学习练习\flower_photos\flower_photos'
dataset = datasets.ImageFolder(root=data_dir, transform=transform5)
# 划分数据集
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=6, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=6, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
class Bottleneck(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, 4*growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4*growth_rate)
        self.conv2 = nn.Conv2d(4*growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat((x, out), 1)
        return out


class Transition(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)

    def forward(self, x):
        out = self.conv(F.relu(self.bn(x)))
        out = F.avg_pool2d(out, 2)
        return out


class DenseNet(nn.Module):
    def __init__(self, block, nblocks, growth_rate=12, reduction=0.5, num_classes=5):
        super(DenseNet, self).__init__()
        self.growth_rate = growth_rate

        num_init_features = 2 * growth_rate
        self.conv1 = nn.Conv2d(3, num_init_features, kernel_size=3, padding=1, bias=False)

        self.dense1 = self._make_dense_layers(block, num_init_features, nblocks[0])
        num_features = num_init_features + nblocks[0] * growth_rate
        out_channels = int(num_features * reduction)
        self.trans1 = Transition(num_features, out_channels)

        num_features = out_channels
        self.dense2 = self._make_dense_layers(block, num_features, nblocks[1])
        num_features += nblocks[1] * growth_rate
        out_channels = int(num_features * reduction)
        self.trans2 = Transition(num_features, out_channels)

        num_features = out_channels
        self.dense3 = self._make_dense_layers(block, num_features, nblocks[2])
        num_features += nblocks[2] * growth_rate
        out_channels = int(num_features * reduction)
        self.trans3 = Transition(num_features, out_channels)

        num_features = out_channels
        self.dense4 = self._make_dense_layers(block, num_features, nblocks[3])
        num_features += nblocks[3] * growth_rate

        self.bn = nn.BatchNorm2d(num_features)
        self.linear = nn.Linear(num_features, num_classes)

    def _make_dense_layers(self, block, in_channels, nblock):
        layers = []
        for i in range(nblock):
            layers.append(block(in_channels, self.growth_rate))
            in_channels += self.growth_rate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.trans3(self.dense3(out))
        out = self.dense4(out)
        out = F.avg_pool2d(F.relu(self.bn(out)),kernel_size=out.size()[2:])
        #print(f"After avg_pool2d, out shape: {out.shape}")
        out = out.view(out.size(0), -1)
        #print(f"After view, out shape: {out.shape}") 
        out = self.linear(out)
        return out


def DenseNet121():
    return DenseNet(Bottleneck, [6,12,24,16], growth_rate=32)




In [9]:
from torchsummary import summary
# 使用summary函数查看模型的详细信息，这里假设输入尺寸为(3, 224, 224)
model_densenet = DenseNet121().to(device)
summary(model_densenet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,728
       BatchNorm2d-2         [-1, 64, 224, 224]             128
            Conv2d-3        [-1, 128, 224, 224]           8,192
       BatchNorm2d-4        [-1, 128, 224, 224]             256
            Conv2d-5         [-1, 32, 224, 224]          36,864
        Bottleneck-6         [-1, 96, 224, 224]               0
       BatchNorm2d-7         [-1, 96, 224, 224]             192
            Conv2d-8        [-1, 128, 224, 224]          12,288
       BatchNorm2d-9        [-1, 128, 224, 224]             256
           Conv2d-10         [-1, 32, 224, 224]          36,864
       Bottleneck-11        [-1, 128, 224, 224]               0
      BatchNorm2d-12        [-1, 128, 224, 224]             256
           Conv2d-13        [-1, 128, 224, 224]          16,384
      BatchNorm2d-14        [-1, 128, 2

In [4]:
# 创建模型实例
model_densenet = DenseNet121()
model_densenet = model_densenet.to(device)

# 设置损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_densenet.parameters(), lr=0.0001)

# 训练模型
model_densenet.train()
num_epochs = 40

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_densenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    print(f"Epoch {epoch+1}, Train Loss: {running_loss / len(train_loader)}")

    # 在验证集上评估模型
    model_densenet.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_densenet(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {accuracy:.2f}%')

    # 将准确率存储到 model_accuracies 字典中
    model_accuracies['densenet'].append(accuracy)

    # 重新设定模型为训练模式
    model_densenet.train()

Epoch 1, Train Loss: 1.2361697893969867
Epoch [1/40], Validation Accuracy: 61.31%
Epoch 2, Train Loss: 1.0899356583551485
Epoch [2/40], Validation Accuracy: 62.67%
Epoch 3, Train Loss: 1.0241139536001245
Epoch [3/40], Validation Accuracy: 65.67%
Epoch 4, Train Loss: 0.9983480230886109
Epoch [4/40], Validation Accuracy: 62.53%
Epoch 5, Train Loss: 0.9529011021767344
Epoch [5/40], Validation Accuracy: 67.30%
Epoch 6, Train Loss: 0.9017110862902232
Epoch [6/40], Validation Accuracy: 70.71%
Epoch 7, Train Loss: 0.8920011225403571
Epoch [7/40], Validation Accuracy: 72.34%
Epoch 8, Train Loss: 0.8507806403296334
Epoch [8/40], Validation Accuracy: 74.25%
Epoch 9, Train Loss: 0.7992302098140425
Epoch [9/40], Validation Accuracy: 75.89%
Epoch 10, Train Loss: 0.7562685747079704
Epoch [10/40], Validation Accuracy: 71.66%
Epoch 11, Train Loss: 0.749924975618416
Epoch [11/40], Validation Accuracy: 69.07%
Epoch 12, Train Loss: 0.727568082739504
Epoch [12/40], Validation Accuracy: 73.16%
Epoch 13, Tr

In [5]:
# 保存模型权重
torch.save(model_densenet.state_dict(), 'densenet_weights.pth')
# 保存模型结构和权重
torch.save(model_densenet, 'densenet_model.pth')
accuracies_list = model_accuracies['densenet']

# 创建一个包含epoch编号和对应准确率的字典
data = {'Epoch': list(range(1, len(accuracies_list) + 1)),
        'Accuracy': accuracies_list}

# 将字典转换为DataFrame
df = pd.DataFrame(data)

# 现在你可以将DataFrame保存为Excel文件
df.to_excel('densenet_accuracies.xlsx', index=False)
# 清理内存
del model_densenet
del inputs
del labels
torch.cuda.empty_cache()

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# 实例化模型
model = DenseNet121() .to(device)

# 加载模型权重
model.load_state_dict(torch.load('densenet_weights.pth'))

# 设置为评估模式
model.eval()

Using device: cuda


DenseNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (dense1): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): Bottleneck(
      (bn1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (2): Bottleneck(
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [11]:
import torch
import csv
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# 确保模型处于评估模式
model.eval()

# 初始化用于存储预测和真实标签的列表
all_preds = []
all_labels = []

# 不计算梯度进行预测
with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        # 将数据移动到GPU，如果可用
        inputs, labels = inputs.to(device), labels.to(device)

        # 前向传播
        outputs = model(inputs)

        # 获取预测概率最高的类别
        _, predicted = torch.max(outputs.data, 1)

        # 将预测结果和真实标签添加到列表中
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 计算准确率、精确率、召回率和F1分数
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)

# 将结果保存到CSV文件
results = {
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
}

# 写入CSV文件
with open('DenseNet评估.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Metric', 'Value'])
    for metric, value in results.items():
        writer.writerow([metric, value])

print("Results have been saved to DenseNet评估.csv")

Evaluating: 100%|██████████| 123/123 [00:12<00:00, 10.03it/s]

Results have been saved to DenseNet评估.csv
